In [1]:
from itertools import product
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x64e7980


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(12)

In [4]:
# df = ROOT.RDF.Experimental.FromSpec("test-spec-nometa.json")
# df = ROOT.RDF.Experimental.FromSpec("test-spec.json")
df = ROOT.RDF.Experimental.FromSpec("data/specs/miniDST-no-higgs-spec.json")

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x84bed10


In [5]:
%%cpp
std::string getProcessName(const std::string& path) {
    std::size_t start_pos = path.find(".P") + 2;
    std::size_t end_pos = path.find(".", start_pos);
    return path.substr(start_pos, end_pos - start_pos);
}

In [6]:
# df = df.DefinePerSample("process_name", "rdfsampleinfo_.GetSampleName()")
df = df.DefinePerSample("process_name", "getProcessName(rdfsampleinfo_.AsString())")

In [7]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")
# df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")
df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPFloatKeys, GPFloatValues); return par;")

float_params = ["Energy", "WWCategorisation.mInv", "WWCategorisation.missE", "WWCategorisation.misspT", "beamPol1", "beamPol2", "crossSection", "crossSectionError"]
# need to get rid of . in RDF column names
def parse_par(par):
    return par.replace(".", "_")

for fp in float_params:
    df = df.Define(f"params_{parse_par(fp)}", f"Parameters.get<float>(\"{fp}\").value()")

In [8]:
df = df.Define("lumi", "1.0 / params_crossSection")

In [9]:
processes = {
    "4f": [
        "sw_l",
        "sw_sl",
        "sze_l",
        "sze_sl",
        "szeorsw_l",
        "sznu_l",
        "ww_h",
        "ww_l",
        "ww_sl",
        "zz_h",
        "zz_l",
        "zz_sl",
        "zznu_l",
        "zznu_sl",
        "zzorww_h",
        "zzorww_l"
        ],
    "6f": [a + b for a, b in product(["ee", "ll", "vv"], ["xxxx", "xyyx", "yyyy"])],
    "2f": [
        "z_eehiq",
        "z_h",
        "z_l"
    ],
    "ae_3f": [
        "w_h", "w_l", "z_h", "z_l", "z_nu",
    ],
    "ea_3f": [
        "w_h", "w_l", "z_h", "z_l", "z_nu",
    ]
}

process_names = []
for pre in processes:
    for sub in processes[pre]:
        process_names.append(f"{pre}_{sub}")

print(process_names)

['4f_sw_l', '4f_sw_sl', '4f_sze_l', '4f_sze_sl', '4f_szeorsw_l', '4f_sznu_l', '4f_ww_h', '4f_ww_l', '4f_ww_sl', '4f_zz_h', '4f_zz_l', '4f_zz_sl', '4f_zznu_l', '4f_zznu_sl', '4f_zzorww_h', '4f_zzorww_l', '6f_eexxxx', '6f_eexyyx', '6f_eeyyyy', '6f_llxxxx', '6f_llxyyx', '6f_llyyyy', '6f_vvxxxx', '6f_vvxyyx', '6f_vvyyyy', '2f_z_eehiq', '2f_z_h', '2f_z_l']


In [10]:
fdf = {}
counts = {}
lumi = {}
pols = list(product(["-1", "1"], repeat=2))
for p, pol in product(process_names, pols):
    e_pol, p_pol = pol
    key = f"{p}_{e_pol}_{p_pol}"
    fdf[key] = df.Filter(f"process_name == \"{p}\"").Filter(f"params_beamPol1 == {e_pol}").Filter(f"params_beamPol2 == {p_pol}")
    lumi[key] = fdf[key].Sum("lumi")

In [13]:
L_int = {}

for key in fdf:
    l = lumi[key].GetValue()
    L_int[key] = l
    if (l > 0.0):
        print(f"{key}: {l}")


4f_sw_l_-1_-1: 443.9473515585022
4f_sw_l_-1_1: 85.09719980599849
4f_sw_l_1_-1: 1444.6390580309153
4f_sw_l_1_1: 614.3360558864907
4f_sw_sl_-1_-1: 598.3264710809962
4f_sw_sl_-1_1: 100.83772646244388
4f_sw_sl_1_-1: 4009.405181776853
4f_sw_sl_1_1: 696.6100695082827
4f_sze_l_-1_-1: 53.390269049937366
4f_sze_l_-1_1: 78.86070343070482
4f_sze_l_1_-1: 43.063118853940296
4f_sze_l_1_1: 53.293164535793025
4f_sze_sl_-1_-1: 149.5025181687616
4f_sze_sl_-1_1: 1072.991968784247
4f_sze_sl_1_-1: 262.09681601249656
4f_sze_sl_1_1: 149.32588675251878
4f_szeorsw_l_-1_-1: 1043.6089224613665
4f_szeorsw_l_-1_1: 507.7655724172963
4f_szeorsw_l_1_-1: 5133.574226380498
4f_szeorsw_l_1_1: 1043.6739054132304
4f_sznu_l_-1_1: 5124.884655687451
4f_sznu_l_1_-1: 4203.232747890091
4f_ww_h_-1_1: 6.726569041202469
4f_ww_h_1_-1: 437.06570596034055
4f_ww_l_-1_1: 100.4210026609367
4f_ww_l_1_-1: 2236.674537654601
4f_ww_sl_-1_1: 6.262266090146128
4f_ww_sl_1_-1: 705.6044561345299
4f_zz_h_-1_1: 1295.7454359596502
4f_zz_h_1_-1: 1192.